# SQL Country Club Database Analysis
## Springboard Data Science Program - SQL Mini Project

This notebook contains solutions to the SQL mini project case study. The project is divided into two parts:

- **Part 1 (Q1-Q9)**: Queries designed for PHPMyAdmin interface
- **Part 2 (Q10-Q13)**: Queries for local SQLite database

### Database Schema Overview

The **country_club** database contains three main tables:
1. **Bookings**: Records of facility reservations
2. **Facilities**: Information about club facilities and costs
3. **Members**: Member details and contact information

---

In [ ]:
# Import Required Libraries
import sqlite3
import pandas as pd
from IPython.display import display, Markdown
import warnings
warnings.filterwarnings('ignore')

## Database Setup and Exploration

First, let's establish a connection to our SQLite database and explore its structure to understand the data we're working with.

In [ ]:
# Database connection and exploration functions
def create_connection(db_file):
    """Create a database connection to the SQLite database"""
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(f"Error connecting to database: {e}")
        return None

def execute_query(conn, query, description=""):
    """Execute a SQL query and return results as a DataFrame"""
    try:
        df = pd.read_sql_query(query, conn)
        if description:
            display(Markdown(f"**{description}**"))
        display(df)
        return df
    except Exception as e:
        print(f"Error executing query: {e}")
        return None

# Connect to the database
conn = create_connection('sqlite_db_pythonsqlite.db')
if conn:
    print("✅ Successfully connected to the database!")
else:
    print("❌ Failed to connect to the database")

In [ ]:
# Explore database structure
display(Markdown("### Database Tables"))
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables_df = execute_query(conn, tables_query, "Available Tables:")

print("\n" + "="*50)
for table in ['Facilities', 'Members', 'Bookings']:
    display(Markdown(f"### {table} Table Structure"))
    schema_query = f"PRAGMA table_info({table});"
    schema_df = execute_query(conn, schema_query)
    
    display(Markdown(f"### Sample Data from {table}"))
    sample_query = f"SELECT * FROM {table} LIMIT 3;"
    sample_df = execute_query(conn, sample_query)

---
## Part 1: PHPMyAdmin Queries (Q1-Q9)

The following queries were designed to be executed in the PHPMyAdmin interface, but we'll run them locally for demonstration. These queries cover fundamental SQL concepts including filtering, aggregation, joins, and conditional logic.

### Q1: Facilities that Charge Member Fees

**Objective:** Identify facilities that charge a fee to members (membercost > 0)

**SQL Concept:** Basic WHERE clause filtering with comparison operators

In [ ]:
# Q1: Facilities that charge a fee to members
q1_query = """
SELECT name 
FROM Facilities 
WHERE membercost > 0;
"""

execute_query(conn, q1_query, "Q1 Results: Facilities that charge member fees")

### Q2: Count of Free Facilities

**Objective:** Count how many facilities do not charge member fees

**SQL Concept:** Using COUNT() aggregate function with WHERE clause filtering

In [ ]:
# Q2: How many facilities do not charge a fee to members?
q2_query = """
SELECT COUNT(*) as free_facilities_count
FROM Facilities 
WHERE membercost = 0;
"""

execute_query(conn, q2_query, "Q2 Results: Number of free facilities for members")

### Q3: Affordable Facilities Analysis

**Objective:** Find facilities where member cost is less than 20% of monthly maintenance cost

**SQL Concept:** Mathematical operations in WHERE clauses, percentage calculations

In [ ]:
# Q3: Facilities where member fee < 20% of maintenance cost
q3_query = """
SELECT facid, 
       name, 
       membercost, 
       monthlymaintenance,
       ROUND((membercost / monthlymaintenance * 100), 2) as cost_percentage
FROM Facilities 
WHERE membercost > 0 
AND membercost < (monthlymaintenance * 0.2);
"""

execute_query(conn, q3_query, "Q3 Results: Facilities with member cost < 20% of maintenance")

### Q4: Specific Facility Details

**Objective:** Retrieve details for facilities with ID 1 and 5 without using OR operator

**SQL Concept:** Using IN operator as alternative to OR conditions

In [ ]:
# Q4: Retrieve details of facilities with ID 1 and 5 (without OR)
q4_query = """
SELECT * 
FROM Facilities 
WHERE facid IN (1, 5);
"""

execute_query(conn, q4_query, "Q4 Results: Details for facilities 1 and 5")

### Q5: Facilities Cost Classification

**Objective:** Label facilities as 'cheap' or 'expensive' based on maintenance costs

**SQL Concept:** CASE statements for conditional logic and data categorization

In [ ]:
# Q5: Classify facilities as cheap or expensive
q5_query = """
SELECT name, 
       monthlymaintenance,
       CASE 
           WHEN monthlymaintenance > 100 THEN 'expensive'
           ELSE 'cheap'
       END AS cost_category
FROM Facilities
ORDER BY monthlymaintenance DESC;
"""

execute_query(conn, q5_query, "Q5 Results: Facilities classified by maintenance cost")

### Q6: Most Recent Member Registration

**Objective:** Find the most recently registered member(s) without using LIMIT

**SQL Concept:** Subqueries with aggregate functions (MAX) for finding extreme values

In [ ]:
# Q6: Get the most recently joined member(s) without LIMIT
q6_query = """
SELECT firstname, 
       surname,
       joindate
FROM Members 
WHERE joindate = (SELECT MAX(joindate) FROM Members)
AND memid != 0;
"""

execute_query(conn, q6_query, "Q6 Results: Most recently joined member(s)")

### Q7: Tennis Court Usage Analysis

**Objective:** Find all members who have used tennis courts with court and member names

**SQL Concept:** Multi-table JOINs, string pattern matching with LIKE, DISTINCT for removing duplicates

In [ ]:
# Q7: Members who have used tennis courts
q7_query = """
SELECT DISTINCT f.name AS court_name,
       (m.firstname || ' ' || m.surname) AS member_name
FROM Bookings b
INNER JOIN Facilities f ON b.facid = f.facid
INNER JOIN Members m ON b.memid = m.memid
WHERE f.name LIKE '%Tennis Court%'
AND m.memid != 0
ORDER BY member_name;
"""

execute_query(conn, q7_query, "Q7 Results: Members who have used tennis courts")

### Q8: High-Value Bookings Analysis (Without Subqueries)

**Objective:** Find bookings on 2012-09-14 costing more than $30, considering different rates for members vs guests

**SQL Concept:** Complex CASE statements, date filtering, cost calculations with differential pricing

In [ ]:
# Q8: Expensive bookings on 2012-09-14 (without subqueries)
q8_query = """
SELECT f.name AS facility_name,
       (m.firstname || ' ' || m.surname) AS member_name,
       CASE 
           WHEN b.memid = 0 THEN b.slots * f.guestcost
           ELSE b.slots * f.membercost
       END AS cost
FROM Bookings b
INNER JOIN Facilities f ON b.facid = f.facid
INNER JOIN Members m ON b.memid = m.memid
WHERE DATE(b.starttime) = '2012-09-14'
AND (
    (b.memid = 0 AND b.slots * f.guestcost > 30) OR
    (b.memid != 0 AND b.slots * f.membercost > 30)
)
ORDER BY cost DESC;
"""

execute_query(conn, q8_query, "Q8 Results: Expensive bookings on 2012-09-14")

### Q9: High-Value Bookings Analysis (With Subquery)

**Objective:** Same result as Q8, but using subquery approach

**SQL Concept:** Derived tables (subqueries in FROM clause), alternative query structuring methods

In [ ]:
# Q9: Same as Q8, but using a subquery
q9_query = """
SELECT facility_name, member_name, cost
FROM (
    SELECT f.name AS facility_name,
           (m.firstname || ' ' || m.surname) AS member_name,
           CASE 
               WHEN b.memid = 0 THEN b.slots * f.guestcost
               ELSE b.slots * f.membercost
           END AS cost
    FROM Bookings b
    INNER JOIN Facilities f ON b.facid = f.facid
    INNER JOIN Members m ON b.memid = m.memid
    WHERE DATE(b.starttime) = '2012-09-14'
) AS booking_costs
WHERE cost > 30
ORDER BY cost DESC;
"""

execute_query(conn, q9_query, "Q9 Results: Same as Q8, using subquery approach")

---
## Part 2: SQLite Local Queries (Q10-Q13)

The following queries are designed for local SQLite execution and demonstrate advanced SQL concepts including revenue calculations, hierarchical data, and time-series analysis.

### Q10: Low Revenue Facilities

**Objective:** Find facilities with total revenue less than $1000

**SQL Concept:** Revenue calculation with differential pricing, GROUP BY aggregation, HAVING clause filtering

In [ ]:
# Q10: Facilities with total revenue less than 1000
q10_query = """
SELECT f.name AS facility_name,
       SUM(CASE 
           WHEN b.memid = 0 THEN b.slots * f.guestcost
           ELSE b.slots * f.membercost
       END) AS total_revenue
FROM Bookings b
INNER JOIN Facilities f ON b.facid = f.facid
GROUP BY f.facid, f.name
HAVING total_revenue < 1000
ORDER BY total_revenue;
"""

execute_query(conn, q10_query, "Q10 Results: Facilities with revenue < $1000")

### Q11: Member Recommendations Report

**Objective:** Show members and who recommended them, sorted alphabetically

**SQL Concept:** Self-joins to relate table to itself, handling NULL values in recommendations

In [ ]:
# Q11: Members and their recommenders
q11_query = """
SELECT m.surname AS member_surname,
       m.firstname AS member_firstname,
       r.surname AS recommender_surname,
       r.firstname AS recommender_firstname
FROM Members m
LEFT JOIN Members r ON m.recommendedby = r.memid
WHERE m.memid != 0
ORDER BY m.surname, m.firstname;
"""

execute_query(conn, q11_query, "Q11 Results: Members and their recommenders")

### Q12: Member Usage by Facility

**Objective:** Show facility usage by members only (excluding guests)

**SQL Concept:** LEFT JOIN with filtering, excluding guest bookings (memid = 0)

In [ ]:
# Q12: Facility usage by members (excluding guests)
q12_query = """
SELECT f.name AS facility_name,
       COUNT(b.bookid) AS member_usage
FROM Facilities f
LEFT JOIN Bookings b ON f.facid = b.facid AND b.memid != 0
GROUP BY f.facid, f.name
ORDER BY member_usage DESC;
"""

execute_query(conn, q12_query, "Q12 Results: Facility usage by members only")

### Q13: Monthly Facility Usage by Members

**Objective:** Analyze facility usage patterns by month, excluding guests

**SQL Concept:** Date functions (strftime), time-series analysis, multi-level grouping

In [ ]:
# Q13: Monthly facility usage by members
q13_query = """
SELECT f.name AS facility_name,
       strftime('%Y-%m', b.starttime) AS month,
       COUNT(b.bookid) AS member_usage
FROM Facilities f
LEFT JOIN Bookings b ON f.facid = b.facid AND b.memid != 0
WHERE b.starttime IS NOT NULL
GROUP BY f.facid, f.name, strftime('%Y-%m', b.starttime)
ORDER BY facility_name, month;
"""

execute_query(conn, q13_query, "Q13 Results: Monthly facility usage by members")

---
## Summary and Conclusion

This notebook demonstrated comprehensive SQL analysis techniques using the Country Club database:

### Key SQL Concepts Covered:
1. **Basic Filtering**: WHERE clauses with comparison operators
2. **Aggregation**: COUNT, SUM with GROUP BY and HAVING
3. **Conditional Logic**: CASE statements for data categorization
4. **Join Operations**: INNER JOIN, LEFT JOIN for multi-table queries
5. **Subqueries**: Both WHERE clause and FROM clause subqueries
6. **Date Functions**: Date filtering and time-series analysis
7. **String Operations**: Concatenation and pattern matching
8. **Self-Joins**: Relating table to itself for hierarchical data

### Business Insights:
- Facility usage patterns and revenue analysis
- Member behavior and recommendation networks
- Cost structure analysis and pricing optimization
- Temporal trends in facility utilization

This analysis provides a foundation for data-driven decision making in club management and member services.

In [ ]:
# Clean up: Close database connection
if conn:
    conn.close()
    print("✅ Database connection closed successfully!")